### Assignment 2
# Alexnet Model 구현

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.architecture = nn.Sequential(
            nn.Conv2d(3, 96, (11,11), stride=(4,4)),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=(3,3), stride=(2,2)), 
            
            nn.Conv2d(96, 256, (5,5), stride=(1,1), padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=(3,3), stride=(2,2)), 
            
            nn.Conv2d(256, 384, (3,3), stride=(1,1), padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 384, (3,3), stride=(1,1), padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 256, (3,3), stride=(1,1), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=(3,3), stride=(2,2)), 
            
            nn.Flatten(),
            
            nn.Linear(9216, 4096),
            nn.ReLU(True),
            
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            
            nn.Linear(4096, 1000),
        )
        
    def forward(self, x):
        x = self.architecture(x)
        return F.log_softmax(x)

In [ ]:
model = Net()

In [ ]:
from torchsummary import summary

summary(model,(3,227,227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
          Flatten-14                 [-

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


전반적인 layer 구조를 살펴보면, Convolution layer와 maxpooling layer가 겹겹이 쌓여있고 flatten 이후에 linear (Full Connected Layer)로 구성되어있다.

Maxpooling layer와 마지막 softmax를 제외한 모든 layer에서는 ReLU activation 함수를 통해 output값을 내준다. 

output shape을 살펴보면, 넓었던 input shape에서 점차 좁고 길어진다는 것을 알 수 있다. shape이 길어질수록 학습해야하는 parameter도 많아지는 것을 발견할 수 있다.

parameter를 살펴보면 Convolution layer와 Linear layer에만 존재하는 것을 알 수 있는데, 이는 Maxpooling과 ReLU에서는 업데이트할 가중치가 존재하지 않기 때문이다.

또한 Flat한 Linear layer의 parameter가 Convolution layer에 비해 압도적으로 많은 것을 알 수 있는데, 이것이 CNN이 적은 parameter로도 공간정보를 잘 활용한다는 반증이다.